# Import Libraries

In [2]:
import findspark                                      #Import library to Search for Spark Installation

findspark.init()

from pyspark.sql import SparkSession                          #Import of Spark Session
from pyspark import SparkContext as spark                     #Import the Regular Spark Contex
from pyspark.sql import SQLContext                            #Import the SQL Spark Contex
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark = SparkSession.builder.getOrCreate()

import os

sc = spark.sparkContext

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1569927633243_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-------------

# Creating Variables

## Buckets

In [ ]:
EY_BUCKET = os.environ['EY_BUCKET']

In [ ]:
GERDAU_BUCKET = os.environ['GERDAU_BUCKET']

## Input Paths

In [ ]:
GERDAU_BUCKET_HANA_OLD_INPUT_PATH = "global/hana/cvo2c_priceconditionsbr_lp/part-00000-cf93149c-1fe6-4d60-9086-7d1d4c6918ce-c000.snappy.parquet"

In [ ]:
GERDAU_BUCKET_HANA_INPUT_PATH = "global/hana/cvo2c_priceconditionsbr_lp"

In [ ]:
GERDAU_BUCKET_ADRC_INPUT_PATH = "global/masterdata/adrc"

In [ ]:
GERDAU_BUCKET_KNA1_INPUT_PATH = "global/masterdata/kna1"

In [ ]:
EY_BUCKET_REGIONS_INPUT_PATH = "SPG_DIMENSIONS/SPG_REGIOES/SPG_RG_REGIOES/SPG_RG_REGIOES.parquet"

## Output Paths

In [ ]:
EY_BUCKET_HANA_OUTPUT_PATH = "SPG_FACTS/SPG_HANA/SPG_HANA.parquet"

----------------

# Importing Tables

In [3]:
# Reading parquet file from the Gerdau Analytics Data Lake
Hana_old = spark.read.parquet("s3://"+GERDAU_BUCKET+"/"+GERDAU_BUCKET_HANA_OLD_INPUT_PATH).drop('year').drop('month').drop('day')   #Billing Hana 
Hana = spark.read.parquet("s3://"+GERDAU_BUCKET+"/"+GERDAU_BUCKET_HANA_INPUT_PATH).drop('year').drop('month').drop('day')   #Billing Hana 
Hana = Hana.unionByName(Hana_old)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Reading Clients Address parquet file from the Gerdau Analytics Data Lake
adrc = spark.read.parquet("s3://"+GERDAU_BUCKET+"/"+GERDAU_BUCKET_ADRC_INPUT_PATH)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Reading Clients parquet file from the Gerdau Analytics Data Lake
kna1 = spark.read.parquet("s3://"+GERDAU_BUCKET+"/"+GERDAU_BUCKET_KNA1_INPUT_PATH)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# SPG Regions
df_region = spark.read.parquet("s3://"+EY_BUCKET+"/"+EY_BUCKET_REGIONS_INPUT_PATH)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

--------------------------

# Creating Defined Functions

In [7]:
def normalizing_date_hifen(col_name):
    removed_array = [('-', '/')]
    r = col_name
    for a, b in removed_array:
        r = regexp_replace(r, a, b)
    return r

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Replacing dots by /
def normalizing_date_dots(col_name):
    removed_chars = (".")
    regexp = "|".join('\{0}'.format(i) for i in removed_chars)
    return regexp_replace(col_name, regexp, "/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
def normalizing_orthography(col_name):
    removed_array = [('Á', 'A'), ('Ã', 'A'),('À', 'A'),('Â', 'A'),('Ä', 'A'),
                    ('É', 'E'),('È', 'E'),('Ê', 'E'),('Ë', 'E'),
                    ('Í', 'I'),('Ì', 'I'),('Î', 'I'),('Ï', 'I'),
                    ('Ó', 'O'),('Õ', 'O'), ('Ò', 'O'),('Ô', 'O'),('Ö', 'O'),
                    ('Ú', 'U'),('Ù', 'U'),('Û', 'U'),('Ü', 'U'),
                    ('Ç', 'C'),('`', ''),("'", ''),('-', ''),('/', ''),(' ', '')]
    r = col_name
    for a, b in removed_array:
        r = regexp_replace(r, a, b)
    return r

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-------------------

# Preparing Regions Table

In [10]:
# Normalizing the column IBGE_SAP_CITY_NAME
df_region=df_region.withColumn("UPPER_NAME", normalizing_orthography(upper(df_region["IBGE_SAP_CITY_NAME"])))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Selecting Necessary Columns
df_region=df_region.select(df_region.BRANCH.alias("BRANCH_REGION")
                          ,df_region.UPPER_NAME.alias("CITY_drop")
                          ,df_region.IBGE_UF_ACRONYMS.alias("STATE_drop"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

---------------

# Preparing Clients (KNA1)

In [12]:
kna1=kna1.select(kna1.KUNNR, kna1.ADRNR, kna1["LAND1"], kna1.DATA_LOAD_TIMESTAMP_PARQUET)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
kna1=kna1.orderBy(kna1["LAND1"]).filter(kna1["LAND1"].like("%BR%"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# Replacing the existing column with a corrected one
for col_name in ["DATA_LOAD_TIMESTAMP_PARQUET"]:
    kna1 = kna1.withColumn(col_name, normalizing_date_hifen(col_name))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
kna1 = kna1.orderBy("DATA_LOAD_TIMESTAMP_PARQUET", ascending=False).distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
kna1 = kna1.drop("DATA_LOAD_TIMESTAMP_PARQUET", "LAND1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

----------------

# Preparing Address (ADRC)

In [17]:
adrc=adrc.select(adrc.ADDRNUMBER, adrc.CITY1, adrc.REGION, adrc["COUNTRY"], adrc.DATA_LOAD_TIMESTAMP_PARQUET)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
adrc=adrc.orderBy(adrc["COUNTRY"]).filter(adrc["COUNTRY"].like("%BR%"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# Replacing the existing column with a corrected one
for col_name in ["DATA_LOAD_TIMESTAMP_PARQUET"]:
    adrc = adrc.withColumn(col_name, normalizing_date_hifen(col_name))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
adrc = adrc.orderBy("DATA_LOAD_TIMESTAMP_PARQUET", ascending=False).distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
adrc = adrc.drop("DATA_LOAD_TIMESTAMP_PARQUET", "COUNTRY")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-----------------

# Processing Section 

In [22]:
# Replacing the existing column with a corrected one
for col_name in ["DATA_LOAD_TIMESTAMP_RAW"]:
    Hana = Hana.withColumn(col_name, normalizing_date_hifen(col_name))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# Replacing the existing column with a corrected one
for col_name in ["BILDOCDATE"]:
    Hana = Hana.withColumn(col_name, normalizing_date_dots(col_name))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# Ordering by the most recent Processing Timestamp
Hana=Hana.withColumn('DATA_LOAD_TIMESTAMP_RAW', from_unixtime(unix_timestamp('DATA_LOAD_TIMESTAMP_RAW', 'yyyy/MM/dd')))\
         .withColumn('BILDOCDATE', from_unixtime(unix_timestamp('BILDOCDATE', 'yyyy/MM/dd')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
Hana = Hana.sort("DATA_LOAD_TIMESTAMP_RAW", ascending=False).distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# Get 1 year worth of data
Hana=Hana.where(Hana['BILDOCDATE']>=add_months(current_date(),-24))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# Creating Key using 7 fields
# SalesOrg       -> Organização de Vendas (Sales Organization)
# salesdocitem   -> Item Ordem de Vendas (Sales Order Item)
# salesdocnumber -> Número da Ordem de Vendas (Sales Order)
# SalesDocType   -> Tipo de Ordem de Vendas (Sales Order Type)
# dateyearmonth  -> Data - Ano/Mês (Date - Year/Month)
# BilDocItem     -> Item Fatura (Bill Item)
# BilDocNumber   -> Número da Fatura (Bill number)
Hana = Hana.withColumn('HANA_KEY', concat(Hana['SALESORG']
                                        ,Hana['SALESDOCITEM']
                                        ,Hana['SALESDOCNUMBER'] 
                                        ,Hana['SALESDOCTYPE']
                                        ,Hana['DATEYEARMONTH']
                                        ,Hana['BILDOCITEM']
                                        ,Hana['BILDOCNUMBER']))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
Hana=Hana.join(kna1, Hana.SoldToCustomer.cast(IntegerType())==kna1.KUNNR.cast(IntegerType()), how='left')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
Hana=Hana.withColumn('SoldToCustomer', when(Hana.KUNNR.isNull(), Hana.SoldToCustomer)\
                                            .otherwise(Hana.KUNNR))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
Hana=Hana.join(adrc, Hana.ADRNR.cast(IntegerType())==adrc.ADDRNUMBER.cast(IntegerType()), how='left')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
Hana=Hana.withColumn('ADRNR', when(Hana.ADDRNUMBER.isNull(), Hana.ADRNR)\
                                            .otherwise(Hana.ADDRNUMBER))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
Hana=Hana.withColumn('SoldToCity', when(Hana.ADDRNUMBER.isNull(), Hana.SoldToCity)\
                                            .otherwise(Hana.CITY1))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
Hana=Hana.withColumn('SoldToState', when(Hana.ADDRNUMBER.isNull(), Hana.SoldToState)\
                                            .otherwise(Hana.REGION))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
Hana=Hana.join(df_region, (normalizing_orthography(upper(trim(Hana.SoldToCity)))==upper(trim(df_region.CITY_drop)))&
                          (normalizing_orthography(upper(trim(Hana.SoldToState)))==upper(trim(df_region.STATE_drop))), how="left")\
            .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
Hana=Hana.drop("ADDRNUMBER", "CITY1", "REGION", "KUNNR", "CITY_drop", "STATE_drop")

In [12]:
# Dropping Duplicates using the created Key as Parameter 
ProcessedReport = Hana.dropDuplicates(['HANA_KEY']).distinct()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

--------------------

# Uploading Section

In [ ]:
# Uploading the Data Frame to the Smart Pricing bucket
ProcessedReport.write.parquet("s3a://"+EY_BUCKET+"/"+EY_BUCKET_HANA_OUTPUT_PATH, mode = "overwrite")